In [1]:
import pandas as pd
import numpy as np 
import re
% matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter
    


import spacy
nlp = spacy.load('en_core_web_lg')

C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [2]:
import pickle
import time

In [3]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [5]:
# Update puncuation list in spacy
nlp.vocab["$"].is_punct = True
nlp.vocab["|"].is_punct = True
nlp.vocab["+"].is_punct = True
nlp.vocab["<"].is_punct = True
nlp.vocab[">"].is_punct = True
nlp.vocab["="].is_punct = True
nlp.vocab["^"].is_punct = True
nlp.vocab["`"].is_punct = True
nlp.vocab["~"].is_punct = True

### Load in Models

In [6]:
#trigram = pickle.load(open('trigram2.sav', 'rb'))

In [7]:
np.__version__

'1.16.3'

In [8]:
lr_model = pickle.load(open('./trained_models/lr_model_tfidf.sav', 'rb'))
#infile = open('./trained_models/lr_model_tfidf.sav','rb')
#lr_model = pickle.load(infile)
nb_model = pickle.load(open('./trained_models/nb_model_tfidf.sav','rb'))

rf_model = pickle.load(open('./trained_models/rf_model_tfidf.sav', 'rb'))
#svm_model = pickle.load(open('tfidf_svm_model.sav', 'rb'))
#wv_lr_model = pickle.load(open('wv_lr_model.sav', 'rb'))
#wv_rf_model = pickle.load(open('wv_rf_model.sav', 'rb'))
#wv_svm_model = pickle.load(open('wv_svm_model.sav', 'rb'))

C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator Pipeline from version

In [9]:
lr_model

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__max_df': (0.75, 0.8, 0.85, 0.9, 0.95, 1.0), 'tfidf__min_df': (0.001, 0.01, 0.1), 'lr__penalty': ('l1', 'l2'), 'lr__C': (0.01, 0.1, 1), 'lr__class_weight': (None, 'balanced')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='f1', verbose=0)

### Write Function to Determine Fake vs. Real

In [10]:
article2 = " Women are prohibited from taking selfies with soccer players. Specifically, Iranâ€™s mœmoral committee has banned women from taking selfies with famous soccer players. "

In [11]:
# helper function to eliminate tokens that are pure punctuation, whitespace, or stopword
# can be updated based on desired filtering 

def process_txt(token):
    return token.is_punct or token.is_space or token.is_stop or token.like_num

In [12]:
# function to take array of articles and turn them into nested list of tokens

def lemmatize_txt(array):
    lemma = []
    
    doc = nlp(array)

    lemma.append([n.lemma_ for n in doc if not process_txt(n)])
    
    return lemma

In [13]:
# function to recombine nested list of tokens into full articles 

def lemma_combine(lis):
    parsed_articles = []
    
    for i in range(len(lis)):
        concat_art = ' '.join(lis[i])
        parsed_articles.append(concat_art)
    
    return parsed_articles

In [14]:
def infer_vector(text):
    text = text.split()
    vector = []
    for i in text:
        try:
            vector.append(model.word_vec(i))
        except:
            pass
    return list(np.mean(vector, axis=0))

In [15]:
#pour les articles de test que nous avons X_test c déjà traité 
#mais pour un nouvel article on doit appeler cette méthode process_article
#afin faire lemma et n-grams
def process_article(article):
    article = re.sub('[^\x00-\x7F]+', "", article) #remove non-ascii characters
    article = re.sub('(\r)+', "",  article) #remove newline characters
    article = re.sub(r'@([A-Za-z0-9_]+)', "",  article) # remove twitter handles
    article = re.sub(r"(https|http)\S+", "",  article) # remove hyperlinks

    # tokenize article text
    lem = lemmatize_txt(article)
    #print(lem)
    # Create bi-grams for our text 
    phrases = Phrases(lem)
    bigram = Phraser(phrases)
    bigram_lem = list(bigram[lem])
    # add text to trigram model
    trigram = Phrases(bigram_lem)
   
    trigram.add_vocab(lem)
    trigram_lem = list(trigram[lem])
    
    # recombine tokens
    tri_lem_comb = lemma_combine(trigram_lem)
    return tri_lem_comb
    # format text for w2v model
    #tri_lem = pd.Series([x for x in tri_lem_comb])
    #vec_text = tri_lem.apply(infer_vector)
    #vec_text = pd.DataFrame(list(map(lambda x: list(x), vec_text)))0.

In [16]:
def fake_detector(tri_lem_comb):
    

    # make predictions
    lr_predicted = lr_model.predict(tri_lem_comb)
    nb_predicted = nb_model.predict(tri_lem_comb)
    
    rf_predicted = rf_model.predict(tri_lem_comb)
    #wv_lr_predicted = wv_lr_model.predict(vec_text)
    #wv_rf_predicted = wv_rf_model.predict(vec_text)
    #wv_svm_predicted = wv_svm_model.predict(vec_text)
    y_test_nb.append(nb_predicted)
    y_test_rf.append(rf_predicted)
    y_test_lr.append(lr_predicted)
    print ("TFIDF NB Prediction:" , lr_predicted)
    print ("TFIDF LR Prediction:" , nb_predicted)
    print ("TFIDF RF Prediction:" , rf_predicted)
    #print ("TFIDF SVM Prediction:" , svm_predicted)
    #print ("W2V LR Prediction:" , wv_lr_predicted)
    #print ("W2V RF Prediction:" , wv_rf_predicted)
    #print ("W2V SVM Prediction:" , wv_svm_predicted)


#### Load test data from original example

In [17]:
# Read in article text
fileObject = open('X_test_data.txt','rb')  
articles = pickle.load(fileObject)  

In [18]:
fileObject = open('y_test_data.txt','rb')  
y_test = pickle.load(fileObject)  

#### Load test data from example kaggle
https://www.kaggle.com/snapcrack/all-the-news#articles1.csv

In [19]:
from sklearn import metrics

from sklearn.metrics import confusion_matrix

In [35]:
articles = pd.read_csv('./test_data/fake_real_test.csv',index_col=0)

In [36]:
fake_articles = articles[articles['label'] == 1]

In [37]:
true_articles = articles[articles['label'] == 0]

In [57]:
art = pd.read_csv('./data/test.csv')

In [91]:

art['text'][226]

'  Donald Trump Wins The Presidency In Historic Mandate Victory As Hillary Clinton Concedes Reaction to the prospect of a Trump presidency rippled across the globe, with financial markets abroad falling as American television networks raised the prospect that Mrs. Clinton might lose. Asian markets were trading sharply lower, down around two percentage points, and in the United States, Dow Jones futures were down as much as 600 points in after-hours trading.   The American people have voted, Donald Trump is president, and the world is in shock \n“And he changeth the times and the seasons: he removeth kings, and setteth up kings: he giveth wisdom unto the wise, and knowledge to them that know understanding:” Daniel 2:21 (KJV) Tonight. the God of Abraham, Isaac and Jacob spoke quite loudly. Change like this country has never seen, like the world has never seen, has arrived at our doorstep. President Trump has been elected leader of the free world . People, you better buckle up because thi

In [83]:
fake_articles.shape

(2000, 2)

In [23]:
text1 = str(fake_articles.iloc[:5]['text'])
text1

'2000    Print They should pay all the back all the mon...\n2001    Why Did Attorney General Loretta Lynch Plead T...\n2002    Red State : \\r\\nFox News Sunday reported this ...\n2003    Email Kayla Mueller was a prisoner and torture...\n2004    Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...\nName: text, dtype: object'

In [24]:
pd.set_option('display.max_colwidth', -1)

In [25]:
len(articles)

4000

In [44]:
fake_articles.tail(270)

text  \
3730  Share on Facebook Share on Twitter For those of you who have no clue who Gord Downie is, you are likely either not Canadian, or were born after the 90s. Gord Downie is the front man for the iconic Canadian band, The Tragically Hip . This band lives in the hearts of millions of Canadians and they are something we can basically call our own, as for some unknown reason they never really made it big anywhere else. It was recently announced that Gord Downie had been diagnosed with stage 4 terminal brain cancer. In the true spirit of the band, they decided to do one final farewell tour across Canada so that all of their loyal fans could enjoy the energy, performance, and great music, live on stage, one last time. The Canadian Broadcasting Corporation (CBC) even aired the final show, in the band’s hometown of Kingston, Ontario. The Tragically Hip is such a big part of being Canadian that our Prime Minister, Justin Trudeau, was even there, singing along.   So, by now you may or may not be wondering why I am calling Gord Downie a real Canadian hero. Yes, his music touched the hearts of millions, but it is because of how he has used his voice, time and time again, for the voiceless. Gord Downie stands up for those who are often not heard and advocates to bring awareness to a dark part of Canadian history and issues that are currently plaguing the First Nations people of Canada. At the end of their very last show, Gord Downie didn’t draw attention to his own illness, and didn’t ask for money to be raised for cancer charities. Instead, Gord Downie addressed Canada’s new Prime Minister, Justin Trudeau, and vocalized his fears that the Indigenous people of Canada are perhaps in more dire straits today than they have ever been. He said he believed that Trudeau could help bring about meaningful change and called upon all Canadians to “be more mindful of northern affairs.” This meant the world to thousands of Indigenous people across the country, as this performance and announcement was broadcast on live television with millions of people tuning in. Leaders in the Indigenous population say Downie’s words are accurate and thanked him for taking the time out of his final performance to speak up for their community across the country despite his own struggle. The Secret Path From the Secret Path website: “Gord Downie began Secret Path as ten poems incited by the story of Chanie Wenjack, a twelve year-old boy who died fifty years ago on October 22, 1966, in flight from the Cecilia Jeffrey Indian Residential School near Kenora, Ontario, walking home to the family he was taken from over 400 miles away. Gord was introduced to Chanie Wenjack (miscalled “Charlie” by his teachers) by Mike Downie, his brother, who shared with him Ian Adams’ Maclean ’s story from February 6, 1967, “The Lonely Death of Charlie Wenjack.””   The stories Gord’s poems tell were fleshed into the ten songs of Secret Path with producers Kevin Drew and Dave Hamelin. Recording took place over two sessions at the Bathouse in Bath, Ontario, in November and December, 2013. The music features Downie on vocals and guitars, with Drew and Hamelin playing all other instruments, except guest contributions by Charles Spearin (bass), Ohad Benchetrit (lap steel/guitar), Kevin Hearn (piano), and Dave “Billy Ray” Koster (drums). In winter 2014, Gord and Mike brought the recently finished music to comic artist Jeff Lemire for his help illustrating Chanie’s story, bringing him and the many children like him to life. Secret Path acknowledges a dark part of Canada’s history – the long-supressed mistreatment of Indigenous children and families by the Residential school system – with the hope of starting our country on a road to reconciliation.” This ten song album was released by Arts & Crafts accompanied by Leire’seighty-eight page graphic novel published by Simon & Schuster Canada. Secret Path was released on October 18th of this year in a deluxe vinyl and book edition and as a book w

In [27]:
articles.head(10)

text  \
0  KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG). Bukhari said Karachi transporters will cut fares when decrease in CNG prices will be made.                        \r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n\r\r\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [49]:
true_articles.tail(90)

,text,label
1910,"LONDON: England have knack of coming unstuck as they chase a Test series whitewash but they will hope to complete the job against Sri Lanka at Lord´s this week -- despite clear signs of improvement from the tourists.England are 2-0 up with one Test to play, but the match starting on Thursday has more riding on it than it might appear for the hosts after recent series victories ended on a low note.Last year, having won the Ashes, England suffered an innings and 46-run defeat by Australia in the fifth Test at The Oval.There was a similarly heavy reverse in South Africa in January when, 2-0 up with one to play, England lost the fourth Test at Centurion by 280 runs.Although they won the second Test against Sri Lanka in Chester-le-Street by nine wickets, there were warning signs for England during that success.Sri Lanka, having become the first team since New Zealand in 1958 to be dismissed for under 120 in three successive Test innings, made 475 in the second knock in Durham -- including a fine hundred by Dinesh Chandimal.A typically good Lord´s pitch could make life harder still for fast bowlers in London, with the most even contest of the series so far in prospect.<br/> <br/> England´s two biggest doubts concern a pair of players who know Lord´s well as it is their Middlesex home ground.Top-order batsman Nick Compton has been struggling for runs, and fast bowler Steven Finn for wickets.With England having again selected a 12-man squad, it looks as if Compton will get another match at number three.Finn´s place is under threat from uncapped seamer Jake Ball, although England appear ready to retain the Middlesex quick.With England captain Alastair Cook now having completed the burdensome business of getting to 10,000 Test runs, the left-handed opener could be in the mood to make a major score at Lord´s.Meanwhile England new-ball duo James Anderson and Stuart Broad, first and third in the world Test bowler rankings respectively, will look to maintain their standing.Two years ago, Sri Lanka captain Angelo Mathews scored a fine Test century in a thrilling draw at Lord´s where the match went down to the last ball.For this fixture, Sri Lanka may make changes to their side.Stylish wicket-keeper/batsman Kusal Perera -- a late addition to the squad after his drugs ban was overturned by the International Cricket Council because of a faulty lab analysis in Doha -- could end more than six months out by replacing injured all-rounder Milinda Siriwardana (twisted ankle).Fast bowler Shaminda Eranga was reported for a suspect action during the second Test and is understood to have undergone tests at the England and Wales Cricket Board´s training centre at Loughborough University this week.Although Eranga is still available to play, Sri Lanka could give a debut to uncapped 29-year-old left-armer Chaminda Bandara in the absence of the injured Dhammika Prasad and Dushmantha Chameera.",0
1911,"LAHORE: Diana Baig, the talented 20-year-old girl plays for Pakistan´s national team in both cricket and football. She represented the country as one of the ""Girls in Green"" at the recent World Twenty20 tournament in India, in between practicing her penalty shoot-out skills.She grew up playing street cricket and football with other children in the magnificent Hunza Valley, their makeshift arenas ringed by some of the world´s tallest mountains in northern Gilgit-Baltistan.Baig belongs to the Ismaili sect of Shia Islam, which largely freed her from the restrictions placed on other more conservative women in the Muslim country, where her gender is battling for greater freedom.From the streets, Baig began playing in community events and for local teams, and by 2010 she was leading the newly-formed Gilgit-Baltistan women´s team.Two years later, she was selected for Pakistan´s A side, and then as a reserve player for the 2013 World Cup. In 2015, she finally won her first international cap, playing for Pakistan against Bangladesh.Her journey to the 

In [28]:
y_test= articles['label']
articles = articles['text']


In [29]:
def evaluate_model(clf, actual, predicted):
    report = metrics.classification_report(actual, predicted, target_names = ["0", "1"])
    matrix = confusion_matrix(actual, predicted)
    
   
    
    print("Confusion Matrix\n", matrix)
    print("Classification Report:\n", report)
    

In [30]:
y_test_nb = []
y_test_rf = []
y_test_lr = []
y_test_real = []
j=0
import random
while(j<100):
    i = random.choice(range(len(articles)))
    article = process_article(articles[i])
    fake_detector(article)
    
    print('real value is')
    print(y_test[i])
    y_test_real.append(y_test[i])
    j=j+1

TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1

TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [0]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
1
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0


In [31]:
y_test_nb = []
y_test_rf = []
y_test_lr = []
y_test_real = []
j=0
import random
while(j<len(articles)):
    #i = random.choice(range(len(articles)))
    article = process_article(articles[j])
    fake_detector(article)
    
    print('real value is')
    print(y_test[j])
    y_test_real.append(y_test[j])
    j=j+1

TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0

TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [0]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [0]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0

TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0

TFIDF NB Prediction: [0]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [0]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [0]
TFIDF LR Prediction: [1]
TFIDF RF Prediction: [1]
real value is
0
TFIDF NB Prediction: [1]
TFIDF LR Prediction: [0]
TFIDF RF Prediction: [1]
real value is
0


KeyboardInterrupt: 

In [ ]:
evaluate_model(nb_model,y_test_real,y_test_nb)

In [ ]:
#evaluate_model(rf_model,y_test_real,y_test_rf)

In [ ]:
evaluate_model(lr_model,y_test_real,y_test_lr)

### Black Box

In [ ]:
class_names = ['0', '1']
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
print(nb_model.predict_proba([articles[1]]))

In [ ]:
print(rf_model.predict_proba([articles[1]]))

In [ ]:
print(lr_model.predict_proba([articles[1]]))

In [ ]:
idx=7

In [ ]:
print('Document id: %d' % idx)
print('Probability =', nb_model.predict_proba(process_article(articles[idx])))
print('True class: %s' % class_names[y_test[idx]])

In [ ]:
exp = explainer.explain_instance(str(process_article(articles[idx])), nb_model.predict_proba,num_features=200)
exp.as_list()

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
i=0

for article in articles[:10]:
        article = process_article(article)
        fake_detector(article) 
        print('real value is')
        print(y_test[i])
        i=i+1